In [2]:
import pandas as pd
import pickle
import numpy as np
import re

In [3]:
def configurations():
    # funciton to generate dictionary for lookup of document settings
    document = {'1': {'URI_Doc':'yes', 'Lit_Doc': 'no', 'Mix_Doc': 'no', 'Mix_Type': None, 'Embed_Out_URI': 'yes', 'Embed_Out_Words': 'no', 'walker': [], 'walk_depth': [] },
                '2': {'URI_Doc':'yes', 'Lit_Doc': 'yes', 'Mix_Doc': 'no', 'Mix_Type': None, 'Embed_Out_URI': 'yes', 'Embed_Out_Words': 'no', 'walker': [], 'walk_depth': []},
                '3': {'URI_Doc':'yes', 'Lit_Doc': 'yes', 'Mix_Doc': 'no', 'Mix_Type': None, 'Embed_Out_URI': 'no', 'Embed_Out_Words': 'yes', 'walker': [], 'walk_depth': []},
                '4': {'URI_Doc':'yes', 'Lit_Doc': 'yes', 'Mix_Doc': 'no', 'Mix_Type': None, 'Embed_Out_URI': 'yes', 'Embed_Out_Words': 'yes', 'walker': [], 'walk_depth': []},
                '5': {'URI_Doc':'yes', 'Lit_Doc': 'yes', 'Mix_Doc': 'yes', 'Mix_Type': 'random', 'Embed_Out_URI': 'yes', 'Embed_Out_Words': 'no', 'walker': [], 'walk_depth': []},
                '6': {'URI_Doc':'yes', 'Lit_Doc': 'yes', 'Mix_Doc': 'yes', 'Mix_Type': 'random', 'Embed_Out_URI': 'no', 'Embed_Out_Words': 'yes', 'walker': [], 'walk_depth': []},
                '7': {'URI_Doc':'yes', 'Lit_Doc': 'yes', 'Mix_Doc': 'yes', 'Mix_Type': 'random', 'Embed_Out_URI': 'yes', 'Embed_Out_Words': 'yes', 'walker': [], 'walk_depth': []},
                '8': {'URI_Doc':'yes', 'Lit_Doc': 'yes', 'Mix_Doc': 'yes', 'Mix_Type': 'all', 'Embed_Out_URI': 'yes', 'Embed_Out_Words': 'no', 'walker': [], 'walk_depth': []}, 
                '9': {'URI_Doc':'yes', 'Lit_Doc': 'yes', 'Mix_Doc': 'yes', 'Mix_Type': 'all', 'Embed_Out_URI': 'no', 'Embed_Out_Words': 'yes', 'walker': [], 'walk_depth': []},
                '10': {'URI_Doc':'yes', 'Lit_Doc': 'yes', 'Mix_Doc': 'yes', 'Mix_Type': 'all', 'Embed_Out_URI': 'yes', 'Embed_Out_Words': 'yes', 'walker': [], 'walk_depth': []}} 

    # walker type
    walker_type = {'R': 'random', 'WL': 'wl'}
    # walk depth
    walk_depth = [2,3]
    # pretrained PMC word2vec model
    pre_train_model = ['no', 'yes']
    # embedding size
    embedding_size = [100]
    # projection settings 
    projection_settings = ['w3c', 'R', 'R+']
    # reasoning
    reasoning = ['no', 'yes']

    new_dict = {}
    for (key, value) in document.items():
        for (key_2, value_2) in walker_type.items():
            for depth in walk_depth:
                for model in pre_train_model:
                    for size in embedding_size:
                        for setting in projection_settings:
                            for reasoner in reasoning:
                                name = str(key) + '_' + str(key_2) + '_' + str(depth) + '_' + str(model) + '_' + str(size) + '_' + str(reasoner) + '_' + str(setting) 
                                new_dict[name] = value

    new_df = pd.DataFrame.from_dict(new_dict).T

    # walker type
    for key, value in walker_type.items():
        mask = new_df.index.str.contains(key)
        new_df.loc[mask, 'walker'] = value

    # walker depth
    for depth in walk_depth:
        for example in list(new_df.index):
            ind = example.find('_', example.find('_') + 2)
            mask = new_df.index.str[ind+1].str.contains(str(depth))
            new_df.loc[mask, 'walk_depth'] = str(depth)

     # pretrained model
    sets = []
    for example in list(new_df.index):
        ind = [m.start() for m in re.finditer(r"_", example)][2]
        if str(example[ind+1:ind+4]) == 'yes':
            new_pretrain = 'yes'
        else:
            new_pretrain = 'no'
        sets.append(new_pretrain)
    new_df['pre_training'] = sets

    #embedding size
    for size in embedding_size:
        mask = new_df.index.str.contains(str(size))
        new_df.loc[mask, 'embedding_size'] = str(size)

     # reasoning settings
    sets_1 = []
    for example in list(new_df.index):
        ind = example[:example.rfind('_')].rfind('_')
        if str(example[ind+1:ind+4]) == 'yes':
            new_reasoner = 'yes'
        else:
            new_reasoner = 'no'
        sets_1.append(new_reasoner)
    new_df['reasoning'] = sets_1

    #projection settings
    sets_2 = []
    for example in list(new_df.index):
        ind = example.rfind('_')
        new_setting = str(example)[ind+1:] 
        sets_2.append(new_setting)
    
    new_df['projection_setting'] = sets_2

   

    # split df into 20 dataframes to run parallel jobs
    df_split = np.array_split(new_df, 20)
    df = new_df
    return df_split, df

df_split, df = configurations()

In [5]:
df.head(10)

,URI_Doc,Lit_Doc,Mix_Doc,Mix_Type,Embed_Out_URI,Embed_Out_Words,walker,walk_depth,pre_training,embedding_size,reasoning,projection_setting
1_R_2_no_100_no_w3c,yes,no,no,None,yes,no,random,2,no,100,no,w3c
1_R_2_no_100_yes_w3c,yes,no,no,None,yes,no,random,2,no,100,yes,w3c
1_R_2_no_100_no_R,yes,no,no,None,yes,no,random,2,no,100,no,R
1_R_2_no_100_yes_R,yes,no,no,None,yes,no,random,2,no,100,yes,R
1_R_2_no_100_no_R+,yes,no,no,None,yes,no,random,2,no,100,no,R+
1_R_2_no_100_yes_R+,yes,no,no,None,yes,no,random,2,no,100,yes,R+
1_R_2_yes_100_no_w3c,yes,no,no,None,yes,no,random,2,yes,100,no,w3c
1_R_2_yes_100_yes_w3c,yes,no,no,None,yes,no,random,2,yes,100,yes,w3c
1_R_2_yes_100_no_R,yes,no,no,None,yes,no,random,2,yes,100,no,R
1_R_2_yes_100_yes_R,yes,no,no,None,yes,no,random,2,yes,100,yes,R


In [6]:
# get correct settings for gridsearch
final_df = df.loc[(df['projection_setting'] == 'R+') & (df['pre_training'] == 'no') & (df['reasoning'] == 'yes')]

In [7]:
#inspect
final_df

,URI_Doc,Lit_Doc,Mix_Doc,Mix_Type,Embed_Out_URI,Embed_Out_Words,walker,walk_depth,pre_training,embedding_size,reasoning,projection_setting
1_R_2_no_100_yes_R+,yes,no,no,None,yes,no,random,2,no,100,yes,R+
1_R_3_no_100_yes_R+,yes,no,no,None,yes,no,random,3,no,100,yes,R+
1_WL_2_no_100_yes_R+,yes,no,no,None,yes,no,wl,2,no,100,yes,R+
1_WL_3_no_100_yes_R+,yes,no,no,None,yes,no,wl,3,no,100,yes,R+
2_R_2_no_100_yes_R+,yes,yes,no,None,yes,no,random,2,no,100,yes,R+
2_R_3_no_100_yes_R+,yes,yes,no,None,yes,no,random,3,no,100,yes,R+
2_WL_2_no_100_yes_R+,yes,yes,no,None,yes,no,wl,2,no,100,yes,R+
2_WL_3_no_100_yes_R+,yes,yes,no,None,yes,no,wl,3,no,100,yes,R+
3_R_2_no_100_yes_R+,yes,yes,no,None,no,yes,random,2,no,100,yes,R+
3_R_3_no_100_yes_R+,yes,yes,no,None,no,yes,random,3,no,100,yes,R+


In [128]:
# save pair embeddings dictionary as pickle file
with open('/Users/teissherman/Desktop/GitHub/IndividualProject/Project/data/Configurations/configurations.pickle', 'wb') as handle:
    pickle.dump(final_df, handle)

In [129]:
# open configurations to gridsearch
with open('/Users/teissherman/Desktop/GitHub/IndividualProject/Project/data/Configurations/configurations.pickle', 'rb') as f:
    configs = pickle.load(f)

In [130]:
# investigate path names to ensure they are correct
for setting in configs.iloc[[0],:].iterrows():
    path_name = setting[0]
    URI_Doc, Lit_Doc, Mix_Doc, Mix_Type = setting[1][0], setting[1][1], setting[1][2], setting[1][3]
    Embed_Out_URI, Embed_Out_Words = setting[1][4], setting[1][5]
    walker, walk_depth = setting[1][6], setting[1][7]
    pre_train_model, embed_size, reasoning, projection = setting[1][8], setting[1][9], setting[1][10], setting[1][11]
    print(type(Mix_Type))

<class 'NoneType'>
